In [1]:
!wget https://www.gutenberg.org/files/1661/1661-0.txt -O book.txt

--2022-10-26 17:45:58--  https://www.gutenberg.org/files/1661/1661-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607430 (593K) [text/plain]
Saving to: ‘book.txt’

book.txt            100%[===================>] 593.19K  1.71MB/s    in 0.3s    

2022-10-26 17:45:59 (1.71 MB/s) - ‘book.txt’ saved [607430/607430]



In [2]:
text = open('book.txt','r',encoding='utf-8').read()
text = text.lower()

In [3]:
sentences = text.split('\n')

In [4]:
import keras
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
tokenizer = Tokenizer(oov_token='<UNK>')

In [6]:
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index)+1
vocab_size

8915

In [7]:
sequences = tokenizer.texts_to_sequences(sentences)

In [8]:
input_sequences = []
for sequence in sequences:
  for i in range(1,len(sequence)):
    n_gram_sequence = sequence[:i+1]
    input_sequences.append(n_gram_sequence)

In [9]:
max_seq_len = max([len(seq) for seq in input_sequences])
max_seq_len

20

In [10]:
padded_sequences = pad_sequences(input_sequences,maxlen=max_seq_len)

In [11]:
import numpy as np
padded_sequences = np.array(padded_sequences)

In [12]:
x = padded_sequences[:,:-1]
lables = padded_sequences[:,-1]

In [13]:
y = tf.keras.utils.to_categorical(lables,num_classes=vocab_size)

In [14]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding ,Bidirectional, LSTM, Dense

In [15]:
model = Sequential()
model.add(Embedding(vocab_size,100,input_length=max_seq_len-1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size,activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 19, 100)           891500    
                                                                 
 bidirectional (Bidirectiona  (None, 512)              731136    
 l)                                                              
                                                                 
 dense (Dense)               (None, 8915)              4573395   
                                                                 
Total params: 6,196,031
Trainable params: 6,196,031
Non-trainable params: 0
_________________________________________________________________


In [16]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='acc',min_delta=0.001)

In [17]:
model.fit(x,y,epochs=20,verbose=1,batch_size=512)

Epoch 1/20
199/199 [==============================] - 15s 35ms/step - loss: 6.2071 - acc: 0.0839
Epoch 2/20
199/199 [==============================] - 7s 35ms/step - loss: 5.3030 - acc: 0.1429
Epoch 3/20
199/199 [==============================] - 7s 35ms/step - loss: 4.7794 - acc: 0.1710
Epoch 4/20
199/199 [==============================] - 7s 36ms/step - loss: 4.2776 - acc: 0.2002
Epoch 5/20
199/199 [==============================] - 7s 36ms/step - loss: 3.7838 - acc: 0.2418
Epoch 6/20
199/199 [==============================] - 7s 36ms/step - loss: 3.3439 - acc: 0.2927
Epoch 7/20
199/199 [==============================] - 7s 36ms/step - loss: 2.9653 - acc: 0.3461
Epoch 8/20
199/199 [==============================] - 7s 36ms/step - loss: 2.6540 - acc: 0.3968
Epoch 9/20
199/199 [==============================] - 7s 37ms/step - loss: 2.4059 - acc: 0.4400
Epoch 10/20
199/199 [==============================] - 7s 37ms/step - loss: 2.2036 - acc: 0.4765
Epoch 11/20
199/199 [=================

In [18]:
speed_text = "I could not help laughing at the ease with which he explained his process of deduction"
next_words = 100

for _ in range(next_words):
  sequence = tokenizer.texts_to_sequences([speed_text])
  padded = pad_sequences(sequence,maxlen=max_seq_len-1)
  predict_x = model.predict(padded) 
  classes_x = np.argmax(predict_x,axis=1)
  output_word = ''
  for word,index in tokenizer.word_index.items():
    if index == classes_x:
      output_word = word
      break
  speed_text += ' ' + output_word
print(speed_text)

1/1 [==============================] - 0s 20ms/step
I could not help laughing at the ease with which he explained his process of deduction “when i have heard mr holmes that i have scored over you ” he answered ringing the whole bag “the miss questioning glances up in the centre of the room waiting maid took out his coronet and of arthur’s face nosed dressed into a boot lace their care in amid an excessive anderson of smoke these road—249 where represents the morning of the vault or the songs and shouts of solid 1888—i was returning from one to the other pausing only at ten and in the pew of a invention of bicycling etc and the paper in his head and
